The Taxi Data for NYC is stored in Parquet files and this takes up quite a bit of space. For space saving and consistency, we will first use this file to create a usable .csv file from the Taxi data .parquet files. 

In [1]:
import glob
import pandas as pd
import numpy as np

First we will create a glob with all of the files for easy itteration. During testing, it was discovered that 2009, 2010, and 2011-2022 all have different column names. We will pull one file from each year-set to determine the column names to use in the rest of our code.

In [2]:
taxi_data = sorted(glob.glob("Taxi Data\*.parquet"))
# df = pd.read_parquet("Taxi Data\yellow_tripdata_2010-01.parquet")
# df.columns

Originally, we needed to create 3 DataFrames to start with due to these differences. One for 2009 data, one for 2010 data, and a third for 2011-2022. We can accomplish this by looping over the enitre folder and including a try except block in our loop to catch any files with different column names. We needed to do this 3 times to extract all the data we need. The project shifted to only needing 2011-2022 so one time through will accomplish what we need.

In [3]:
temp_list_11_22 = []
col_list_11_22 = ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance',
       'PULocationID', 'DOLocationID','RatecodeID','store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount']

for file in taxi_data:
    try:
        df = pd.read_parquet(file, columns=col_list_11_22).sample(n=100000)
    except Exception as e:
        print(f"{file} Does not include correct cols.")
    else:
        temp_list_11_22.append(df)

Taxi Data\yellow_tripdata_2009-01.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-02.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-03.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-04.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-05.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-06.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-07.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-08.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-09.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-10.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-11.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2009-12.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2010-01.parquet Does not include correct cols.
Taxi Data\yellow_tripdata_2010-02.parquet Does not 

Next, we need to combine all of the data from each file into an overal DataFrame.

In [4]:
overall_data = pd.concat(temp_list_11_22)

With one DataFrame with all the data we need, we can do some inital cleaning that will carry through the CSV format.

We will:
    -Rename the columns to more user friendly names
    -Create a new column for just the year of each transaction for quicker access

In [5]:
cols_rename = ['VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'RateCodeID', 'store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']
overall_data.columns = cols_rename

In [6]:
overall_data['year'] = overall_data["pickup_datetime"].dt.year

In [7]:
overall_data['is_covid_rel'] = np.where(overall_data["year"] > 2019, True, False)

In [8]:
overall_data = overall_data[overall_data.year >= 2011]
overall_data.sort_values(by='year')

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,RateCodeID,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,year,is_covid_rel
1910701,1,2011-01-06 19:11:12,2011-01-06 19:27:17,2.0,3.70,186,141,1.0,N,1,12.1,0.50,0.5,2.62,0.0,0.0,15.72,2011,False
5954211,2,2011-09-13 17:54:00,2011-09-13 18:15:00,2.0,2.44,246,144,1.0,None,1,11.7,1.00,0.5,3.00,0.0,0.0,16.20,2011,False
1984290,2,2011-09-05 13:59:00,2011-09-05 14:26:00,1.0,17.05,28,170,2.0,None,1,45.0,0.00,0.5,5.00,4.8,0.0,55.30,2011,False
1091668,1,2011-09-03 08:21:10,2011-09-03 08:30:07,1.0,2.10,236,142,1.0,N,1,8.1,0.00,0.5,2.00,0.0,0.0,10.60,2011,False
14344570,2,2011-09-30 14:39:00,2011-09-30 15:05:00,1.0,0.83,33,236,1.0,None,2,23.3,0.00,0.5,0.00,0.0,0.0,23.80,2011,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271286,1,2022-04-29 11:52:56,2022-04-29 12:06:22,1.0,3.90,132,219,1.0,N,1,14.5,1.25,0.5,1.50,0.0,0.3,18.05,2022,True
778136,1,2022-04-07 17:17:18,2022-04-07 17:47:31,1.0,3.00,236,164,1.0,N,1,19.0,3.50,0.5,5.00,0.0,0.3,28.30,2022,True
446585,2,2022-04-04 20:47:40,2022-04-04 20:56:31,1.0,1.37,161,237,1.0,N,1,7.5,0.50,0.5,1.70,0.0,0.3,13.00,2022,True
1725153,1,2022-04-15 19:03:12,2022-04-15 19:13:15,4.0,1.40,186,79,1.0,N,2,8.0,3.50,0.5,0.00,0.0,0.3,12.30,2022,True


In [9]:
overall_data['payment_type'] = overall_data['payment_type'].values.astype(str).astype(int)

In [10]:
#change all 1 values to credit card
overall_data['payment_type'] = overall_data['payment_type'].replace(1, 'credit_card')
#change all 2 values to cash
overall_data['payment_type'] = overall_data['payment_type'].replace(2, 'cash')
#change all 3 values to no charge
overall_data['payment_type'] = overall_data['payment_type'].replace(3, 'no_charge')
#change all 4 values to dispute
overall_data['payment_type'] = overall_data['payment_type'].replace(4, 'dispute')
#change all 5 values to unknown
overall_data['payment_type'] = overall_data['payment_type'].replace(5, 'unknown')
#change all 6 values to voided trip
overall_data['payment_type'] = overall_data['payment_type'].replace(6, 'voided_trip')

Now we export to the CSV to be used in our main analysis.

In [11]:
overall_data.to_csv("overall_data.csv", index=False)